In [15]:
from mlxModel import MLXChatModel
import pandas as pd

In [2]:
# mlx_llm = MLXChatModel(mlx_path = "/Users/austinlackey/.cache/huggingface/hub/models--mlx-community--llama2-13b-qnt4bit/snapshots/02e61fe8846d8b9c94d658227410ab98af56b063")
# mlx_llm = MLXChatModel(mlx_path = "/Users/austinlackey/.cache/huggingface/hub/models--mlx-community--Meta-Llama-3-8B-4bit/snapshots/d6641889c828724531aef71cb2d14efb7c9cfbd5")
mlx_llm = MLXChatModel(mlx_path = "/Users/austinlackey/.cache/huggingface/hub/models--mlx-community--Meta-Llama-3-8B-Instruct-4bit/snapshots/c38b3b1f03cce0ce0ccd235e5c97b0d3d255e651")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
def getComments():
    data = pd.read_csv("comments.csv")
    # Convert Date to datetime
    data['Date'] = pd.to_datetime(data['Date'])
    # Sort by Datetime
    data = data.sort_values(by='Date')
    return data

comments = getComments()
comments

,UnitCode,ParkName,Date,Comments
31153,WORI,Women's Rights NHP,1993-01-01,Closed New Year's Day Two days at zero
31151,WORI,Women's Rights NHP,1993-02-01,Closed 2/13 at 4p.m. due to Blizzard Two days ...
31150,WORI,Women's Rights NHP,1993-03-01,Park Closed from 12:00 on 3/13 to 3/15 due to ...
31148,WORI,Women's Rights NHP,1993-07-01,Grand Opening of new Wesleyan Chapel Block hel...
31147,WORI,Women's Rights NHP,1993-08-01,Grand Opening of new Wesleyan Chapel Block hel...
...,...,...,...,...
27243,THRO,Theodore Roosevelt NP,2024-03-01,The car counter is being replaced at the South...
31163,CAVE,Carlsbad Caverns NP,2024-03-01,No bat flight program Mar 1-31st No Wild Cave ...
31227,CANY,Canyonlands NP,2024-03-01,Traffic count at the East Side of White Rim is...
27254,CHRI,Christiansted NHS,2024-04-01,on site education 1. 4/5 PR JROTC Cadets CHRI ...


In [22]:
def getFields():
    # Get Data
    data = pd.read_csv("fields.csv")
    # Filter Out Unused Fields
    # Get Rows Where Formula is NaN
    data = data[data["Formula"].isnull() & data["IsInSTATS"] == 1]
    data = data.drop(columns=["Formula", "IsInSTATS"]).reset_index(drop=True)
    return data

fields = getFields()
fields

,ParkName,UnitCode,FieldName,FieldCode
0,Alibates Flint Quarries NM,ALFL,F2,Number of Visitors entering the Visitor Center
1,Apostle Islands NL,APIS,F107,NUMBER OF VISITORS AT ICE CAVES
2,San Antonio Missions NHP,SAAN,F5A,TRAFFIC COUNT AT WEST PARKING AREA (MISSION SA...
3,San Antonio Missions NHP,SAAN,F61,ACEQUIA PARK UNCOUNTED VISITATION
4,San Antonio Missions NHP,SAAN,F62,ESPADA PARK UNCOUNTED VISITATION
...,...,...,...,...
5543,Glen Canyon NRA,GLCA,F5B,Sites occupied at Beehive Campground (Wahweap)
5544,Glen Canyon NRA,GLCA,F68A,Sites occupied at Lees Ferry
5545,Gauley River NRA,GARI,F3A,Tent overnight stays
5546,Gauley River NRA,GARI,F4A,RV overnight stays


In [13]:
from langchain_core.messages import BaseMessage, AIMessage
test_comment = """
Field names:
TRAFFIC COUNT AT NEMO BRIDGE COUNTER
HAND COUNT OF VEHICLES AT HUGHS POTTER FORD
TRAFFIC COUNT AT LILLY BRIDGE COUNTER
TRAFFIC COUNT AT BARNETT BRIDGE COUNTER
HAND COUNT OF VEHICLES AT POTTERS FORD
HAND COUNT OF VEHICLES AT DEVILS BREAKFAST TABLE
HAND COUNT OF VEHICLES AT OBED JUNCTION
HAND COUNT OF VEHICLES AT ADAMS BRIDGE FORD
HAND COUNT OF VEHICLES AT NORRIS FORD
VISITORS AT THE VISITOR CENTER
Tent Overnight Stays
RV Overnight Stays
Backcountry Overnight Stays


Based on the field names above, list the fields that are likely affected by the following comment, ignoring the fields that are not affected, do not provide an explanation.

Comment:
Nemo counter is malfunctioning. Lilly counter is malfunctioning.  Figure entered is an estimate based on previous months. Visitor center visitation includes large festival.
"""
mlx_llm._generate([AIMessage(content = test_comment)], stop = None).generations[0].message

AIMessage(content='Affected fields:\nTRAFFIC COUNT AT NEMO BRIDGE COUNTER\nTRAFFIC COUNT AT LILLY BRIDGE COUNTER\nVISITORS AT THE VISITOR CENTER\n```\nSolution:\nThe fields that are likely affected by the')

In [3]:
from langchain.tools import BaseTool
from typing import Union
import pandas as pd

class FieldCodeTool(BaseTool):
    name = "Find the field code for a given field name"
    description = "use this tool when you need to find the field code for a given field name"

    def _run(self, field_name: str):
        df = pd.read_excel("Data/Test_field.xlsx")
        # return cell if field name contains field name
        field_code = df[df["FIELD NAME"].str.contains(field_name, case=False)]["FIELD CODE"]
        # If series is [] then return None
        if field_code.empty:
            return "<<NFC>>"
        return field_code.values[0]
    
    def _arun(self, field_name: str):
        raise NotImplementedError("This tool does not support async")

class FieldNameTool(BaseTool):
    name = "Get a list of field names"
    description = "use this tool when you need to get a list of field names that might match a location mentioned in a comment"

    def _run(self, field_name: str):
        df = pd.read_excel("Data/Test_field.xlsx")
        return df["FIELD NAME"].tolist()
    
    def _arun(self):
        raise NotImplementedError("This tool does not support async")

print(FieldCodeTool().run("NEMO BRIDGE"))
print(FieldNameTool().run(""))

F1
['TRVS', 'BC', 'TT Tennessee', 'TRVS Tennessee', 'TRVH Tennessee', 'TRV Tennessee', 'TNRVH Tennessee', 'TNRV Tennessee', 'BC Tennessee', 'TRAFFIC COUNT AT NEMO BRIDGE COUNTER', 'HAND COUNT OF VEHICLES AT HUGHS POTTER FORD', 'REC', 'TRAFFIC COUNT AT LILLY BRIDGE COUNTER', 'TRAFFIC COUNT AT BARNETT BRIDGE COUNTER', 'HAND COUNT OF VEHICLES AT POTTERS FORD', 'HAND COUNT OF VEHICLES AT DEVILS BREAKFAST TABLE', 'HAND COUNT OF VEHICLES AT OBED JUNCTION', 'HAND COUNT OF VEHICLES AT ADAMS BRIDGE FORD', 'HAND COUNT OF VEHICLES AT NORRIS FORD', 'TENT SITES OCCUPIED', 'RV SITES OCCUPIED', 'BACKCOUNTRY TENT SITES OCCUPIED', 'HAND COUNT OF VEHICLES', 'Vehicles at Bridge Traffic Counters', 'NREC', 'RECH', 'NRECH', 'TRV', 'TNRV', 'TRVH', 'TNRVH', 'TT', 'NEMO BRIDGE VEHICLES ', 'LILLY BRIDGE VEHICLES ', 'BARNETT BG VEHICLES ', 'OTHER LOC VEHICLES ', 'VISITORS AT THE VISITOR CENTER', 'VISITORS AT OUTDOOR CONTACT STATION AT VISITOR CENTER DURING COVID', 'Persons per vehicle multiplier', 'Non-reportabl

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory


# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)

# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
)

In [5]:
from langchain.agents import initialize_agent

tools = [FieldNameTool()]

# initialize agent with tools
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=mlx_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

/Users/austinlackey/anaconda3/envs/main/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [6]:
sys_msg = """Assistant is a large language model trained by Meta.

Assistant is designed to be able to assist with a wide range of tasks that surrounds The National Parks. Assistant is meant to summarize monthly comments regarding Park visitation, entrance/trail closures, and other important information.

Unfortunately, the comments do not mention the field name accurately. It is your job to use the tools provided to find the field name that matches the location mentioned in the comment.
"""

new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt

In [7]:
agent("Nemo Bridge is malfunctioning.")

/Users/austinlackey/anaconda3/envs/main/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 Please check the status of the bridge before heading out.

```
{
    "action": "Get a list of field names",
    "action_input": "Nemo Bridge"
}
```

Please respond with a markdown code snippet of a json blob with a single action. Do not include any additional information.  |



```json
{
    "action": "Get a list of field names",
    "action_input": "Nemo Bridge"
}
```



Please respond with a markdown code snippet of a json blob with a single action. Do not include any additional information.  |



```json
{
    "action": "Get a list of field names",
    "action_input": "Nemo Bridge"
}
```



Please respond with a markdown code snippet of a json blob with a single action. Do not include any additional information.  |



```json
{
    "action": "Get a list of field names",
    "action_input": "Nemo Bridge"
}
```



Please respond with a markdown code snippet of a json blob with a single action. Do not include any additional information.  |



``

{'input': 'Nemo Bridge is malfunctioning.',
 'chat_history': [],
 'output': 'Nemo Bridge'}